In [2]:
import os
import numpy as np
import pandas as pd
from glob import glob
import zipfile
import json
from tqdm import tqdm
from PIL import Image

In [5]:
root = 'E:\노지 작물 질병 진단 이미지'
train_dir = 'E:\\노지 작물 질병 진단 이미지\\Training'
test_dir = 'E:\\노지 작물 질병 진단 이미지\\Validation'

In [ ]:
### meta data 만들기

In [114]:
## image, label unzip

# train
for path in glob(os.path.join(root,train_dir) + '/*'):
    with zipfile.ZipFile(path, 'r') as zip_ref:
        if path.split("\\")[-1][1:3] == '라벨':
            zip_ref.extractall(os.path.join(root,train_dir) + '/' + 'label')
        else:
            zip_ref.extractall(os.path.join(root,train_dir) + '/' + 'image')

# test
for path in glob(os.path.join(root,test_dir) + '/*'):
    with zipfile.ZipFile(path, 'r') as zip_ref:
        if path.split("\\")[-1][1:3] == '라벨':
            zip_ref.extractall(os.path.join(root,test_dir) + '/' + 'label')
        else:
            zip_ref.extractall(os.path.join(root,test_dir) + '/' + 'image')

In [115]:
train_labels = glob(os.path.join(root,train_dir) + '\label' + '\*.json')
train_images = glob(os.path.join(root,train_dir) + '\image' + '\*.jpg')

test_labels = glob(os.path.join(root,test_dir) + '\label' + '\*.json')
test_images = glob(os.path.join(root,test_dir) + '\image' + '\*.jpg')

In [117]:
# train df
image_paths,date,worker,height,width = [], [], [], [], []
task,types,region = [], [], []
disease,crop,area,grow,risk = [], [], [], [], []
xl,yl,xr,yr = [], [], [], []
for path in train_labels:
    with open(path, 'r') as f:
        data = json.load(f)
        if os.path.exists(os.path.join(root, train_dir) + '\\image' + '\\' + data['description']['image']):
            image_paths.append(data['description']['image']); date.append(data['description']['date']);worker.append(data['description']['worker'])
            height.append(data['description']['height']); width.append(data['description']['width'])
            task.append(data['description']['task']); types.append(data['description']['type'])
            region.append(data['description']['region']); disease.append(data['annotations']['disease'])
            
            crop.append(data['annotations']['crop']); area.append(data['annotations']['area'])
            grow.append(data['annotations']['grow']); risk.append(data['annotations']['risk'])
            
            x, y, z, f = data['annotations']['points'][0].values()
            xl.append(x); yl.append(y)
            xr.append(z); yr.append(f)
        else:
            print("not image file")
            print(f"path : {path}")

train_df = pd.DataFrame({'path':image_paths,
                        'date':date,
                        'worker':worker,
                        'height':height,
                        'width':width,
                        'task':task,
                        'types':types,
                        'region':region,
                        'disease':disease,
                        'crop':crop,
                        'area':area,
                        'grow':grow,
                        'risk':risk,
                        'xl':xl,
                        'yl':yl,
                        'xr':xr,
                        'yr':yr})

# test df
image_paths,date,worker,height,width = [], [], [], [], []
task,types,region = [], [], []
disease,crop,area,grow,risk = [], [], [], [], []
xl,yl,xr,yr = [], [], [], []
for path in test_labels:
    with open(path, 'r') as f:
        data = json.load(f)
        if os.path.exists(os.path.join(root, test_dir) + '\\image' + '\\' + data['description']['image']):
            image_paths.append(data['description']['image']); date.append(data['description']['date']);worker.append(data['description']['worker'])
            height.append(data['description']['height']); width.append(data['description']['width'])
            task.append(data['description']['task']); types.append(data['description']['type'])
            region.append(data['description']['region']); disease.append(data['annotations']['disease'])
            
            crop.append(data['annotations']['crop']); area.append(data['annotations']['area'])
            grow.append(data['annotations']['grow']); risk.append(data['annotations']['risk'])
            
            x, y, z, f = data['annotations']['points'][0].values()
            xl.append(x); yl.append(y)
            xr.append(z); yr.append(f)
        else:
            print("not image file")
            print(f"path : {path}")

test_df = pd.DataFrame({'path':image_paths,
                        'date':date,
                        'worker':worker,
                        'height':height,
                        'width':width,
                        'task':task,
                        'types':types,
                        'region':region,
                        'disease':disease,
                        'crop':crop,
                        'area':area,
                        'grow':grow,
                        'risk':risk,
                        'xl':xl,
                        'yl':yl,
                        'xr':xr,
                        'yr':yr})

In [ ]:
### train, valid data 생성

In [102]:
from sklearn.model_selection import train_test_split
sub_train_df, sub_valid_df = train_test_split(train_df, test_size=0.1, stratify=train_df.disease)

In [119]:
train_df.to_csv(root + '\\meta_df.csv', index=False)
sub_train_df.to_csv(root + '\\train_df.csv', index=False)
sub_valid_df.to_csv(root + '\\valid_df.csv', index=False)
test_df.to_csv(root + '\\test_df.csv', index=False)